In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/TRVQA

# List files to make sure we're in the expected directory.
# Your output will look different, showing your own Drive files here.
!ls

/content/drive/My Drive/TRVQA
bench_10S_10I		       bench_out_pl
bench_10S_10I_start	       bench_out_shot_diff
benchmark		       bench_out_shot_diff_2436
bench_out		       bench_out_shot_diff_24_real
bench_out_100		       bench_out_shot_diff_36_real
bench_out_1000		       bench_out_shot_diff_4816_real
bench_out_100_old	       bench_out_shot_diff_52
bench_out_150		       bench_out_shot_diff_52_real
bench_out_2436		       bench_tsp_4816
bench_out_4816		       bench_tsp_49
bench_out_4816_lr_3	       circuit.py
bench_out_4816_old	       ex_value_pots
bench_out_4816_sv_compare      gates
bench_out_52		       hamiltonian
bench_out_52_lr		       __init__.py
bench_out_52_memory	       maxcut_comparison_CPU
bench_out_52_old	       maxcut_comparison_GPU
bench_out_52_opt	       measure
bench_out_comparison	       optimization
bench_out_comparison_cpu_real  __pycache__
bench_out_comparison_gpu       utils


In [ ]:
import sys
sys.path.append('/content/drive/My\ Drive/TRVQA')


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-691171030.py:2: SyntaxWarning: invalid escape sequence '\ '
  sys.path.append('/content/drive/My\ Drive/TRVQA')


In [ ]:
!pip install qiskit
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 149.6 MB/s eta 0:00:00


In [ ]:
import sys
import os
import sys
import os
# Add the parent directory of TRVQA to sys.path
from measure.enums import MeasureMethod
from optimization.gradients.vanilla_parameter_shift import vanilla_parameter_shift
from optimization.optimization import minimize, minimize_custom
#from utils.maxcut import gengraph, create_hamiltonian, make_hamiltonian
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

import numpy as np
import torch
from circuit import Circuit

def compute_maxcut_value(bitstring, graph):
    """Computes the MaxCut value for a given bitstring solution."""
    cut_value = sum(w for (u,v), w in graph if bitstring[u] != bitstring[v])
    return cut_value

# for s in best_value:
#     print(compute_maxcut_value(s, g))

from pyparsing import Optional
import torch
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from optimization.optimization import minimize, minimize_custom
# TRVQA imports

from circuit import Circuit
from hamiltonian.hamiltonian import Hamiltonian
from measure.enums import MeasureMethod
torch.cuda.empty_cache()
import numpy as np
import torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
import torch

from measure.enums import MeasureMethod
from optimization.optimization import minimize
from utils.maxcut import gengraph, create_hamiltonian,make_hamiltonian
from hamiltonian.hamiltonian import Hamiltonian
import numpy as np
import torch
from circuit import Circuit
import time


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx

def maxcut_value(G) -> float:
    """
    Solve weighted Max-Cut on an undirected NetworkX graph using Gurobi.

    Args:
        G: networkx.Graph
           Undirected graph. Edge weights read from attribute 'weight' (default 1.0).

    Returns:
        float: Optimal Max-Cut value.
    """
    # if not isinstance(G, nx.Graph) or G.is_directed():
    #     raise ValueError("G must be an undirected networkx.Graph")

    model = gp.Model("MaxCut")
    model.Params.OutputFlag = 0  # silence solver output

    # Binary variable x[i] ∈ {0,1} indicates partition side of node i
    #nodes = list(G.nodes())
    # Number of nodes
    nodes = set()
    for e,w in G:
        u,v = e
        nodes.add(u)
        nodes.add(v)
    n = len(nodes)

    x = model.addVars(nodes, vtype=GRB.BINARY, name="x")
    # Objective: Maximize the cut value
    model.setObjective(
        gp.quicksum(w * (x[u] + x[v] - 2 * x[u] * x[v]) for (u, v), w in G),
        GRB.MAXIMIZE
    )

    # Optimize the model
    model.optimize()


    if model.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Gurobi did not find an optimal solution (status={model.Status}).")

    return float(model.ObjVal)
import json

N_list = [4, 8, 16, 24, 36, 52, 70, 90, 100]
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Callable, Any
import torch, numpy as np

# --- Result container ---------------------------------------------------------
@dataclass
class RunLog:
    exp_values: List[float]
    best_results: List[Any]   # bitstrings, ints, etc.
    iter_times: List[float]
    final_theta: torch.Tensor

# --- Helpers ------------------------------------------------------------------
def build_qiskit_like_hamiltonian(hm):
    """hm is your [(coeff, bool_list), ...]"""
    pauli_strings, coefficients = [], []
    for coeff, bool_list in hm:
        pauli_strings.append(''.join('Z' if b else 'I' for b in bool_list))
        coefficients.append(coeff)
    return Hamiltonian(len(bool_list), pauli_strings, coefficients)

def build_circuit(N: int, D: int, rank: int = 10, device: str = 'cpu', isRing: bool = True) -> Circuit:
    c = Circuit(N, device=device)
    for _ in range(D):
        for j in range(N): c.rx(j)
        for j in range(N): c.rz(j)
        for j in range(N-1): c.cx(j, j+1)
        if isRing:
            c.cx(N-1, 0)
    c.rank = rank
    return c

# --- Method runners -----------------------------------------------------------
def run_statevector_vqe(N: int, D: int, h, iter_steps: int, lr: float = 1e-2, theta0=None, isRing=True) -> RunLog:
    model = QuantumCircuitTorch(theta0, N, D, h, isRing=isRing)
    #opt = torch.optim.RMSprop(model.parameters(), lr =0.05, alpha=0.95, eps=1e-8, weight_decay=0.0)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    exp_vals, best_results, iter_times = [], [], []
    for _ in range(iter_steps):
        start = time.time()
        opt.zero_grad()
        out = model()                   # expectation value (loss)
        out.backward()
        opt.step()
        iter_times.append(time.time() - start)
        exp_vals.append(float(out.item()))
        best_results.append(get_opt_cut_sv(model.theta, N, D, h, isRing))
        # if you track iteration time, append here (e.g., time.perf_counter deltas)
    return RunLog(exp_values=exp_vals, best_results=best_results, iter_times=iter_times, final_theta=model.theta.detach().clone())

def run_tr_vqe(method, circuit, theta0, hamil, iter_steps: int, lr: float, grad_kwargs: dict) -> RunLog:
    """Generic TR-VQE using your Optimizer + ParameterShift flavors."""
    # # Build gradient object with provided kwargs, override measure method:
    from optimization.gradients.batch_parameter_shift import BatchParameterShiftGradient
    from optimization.optimizer import Optimizer
    grad = BatchParameterShiftGradient(
        torch.pi/2,
        grad_kwargs.get("batches", 100),
        grad_kwargs.get("shots", int(1e3)),
        method,                      # MeasureMethod
        grad_kwargs.get("D", 1)
    )
    opt = Optimizer(torch.optim.SGD, {'lr': 0.05})
    #opt = Optimizer(torch.optim.AdamW, {'lr': lr})
    #opt = Optimizer(torch.optim.AdamW, {'lr': lr})
    # opt = Optimizer(torch.optim.RMSprop, {
    #     "lr": 0.05,      # 0.03–0.04 works best at N=8
    #     "alpha": 0.95,     # smoother EWMA than 0.9
    #     "eps": 1e-8,
    #     # "momentum": 0.9,   # NEW
    #     # "centered": True,  # NEW
    #     "weight_decay": 0.0,
    # })
    theta_init = theta0.clone().detach().requires_grad_(True)

    new_theta, exp_vals, best_results, iter_times = minimize(
        circuit, theta_init, hamil, opt, grad, iter_steps, best_value_method="argmax_tr_noinv_LE"
    )
    return RunLog(
        exp_values=[float(v.real) for v in exp_vals],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=new_theta.detach().clone()
    )


In [ ]:
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

def run_circuit_sv(theta,N,D,h, isRing):
    c = QuantumCircuit(N)
    theta_t = 0
    for i in range(D):
        for ind1 in range(N):
            c.rx(theta[theta_t].detach().cpu().item(), ind1)
            theta_t+=1
        for ind1 in range(N):
            c.rz(theta[theta_t].detach().cpu().item(), ind1)
            theta_t+=1
        for ind1 in range(N-1):
            c.cx(ind1, ind1+1)
        if isRing:
            c.cx(N-1,0)  # Add a CX to make it more complex
    sv = Statevector(c).expectation_value(h)
    return sv.real
def get_opt_cut_sv(theta,N,D,h, isRing):
    c = QuantumCircuit(N)
    t = 0
    for i in range(D):
        for j in range(N):
            c.rx(theta[t].detach().cpu().item(), j)
            t+=1
        for j in range(N):
            c.rz(theta[t].detach().cpu().item(), j)
            t+=1
        for j in range(N-1):
            c.cx(j, j+1)
        if isRing:
            c.cx(N-1,0)  # Add a CX to make it more complex
    sv = Statevector(c)
    my_dict = (sv.probabilities_dict())
    max_key = min(my_dict, key=my_dict.get)
    #print("min key" , max_key)
    probabilities = np.abs(sv) ** 2
    # Step 2: Get index of highest probabili
    max_index = probabilities.argmax()
    return format(max_index,f'0{N}b')

# Custom PyTorch autograd function
class QuantumFunction(Function):
    @staticmethod
    def forward(ctx, theta, gates, N, h, D, isRing):
        ctx.save_for_backward(theta)
        ctx.gates = gates
        ctx.N = N
        ctx.D = D
        ctx.h = h
        ctx.isRing = isRing
        output = run_circuit_sv(theta.detach(), N,D,h, isRing)
        return torch.tensor([output], dtype=torch.float32)
    @staticmethod
    def backward(ctx, grad_output):
        theta, = ctx.saved_tensors
        grad = parameter_shift_grad_sv(theta.detach(), ctx.gates, ctx.N, ctx.D, ctx.h, ctx.isRing)
        return grad_output *grad, None, None, None, None, None
# Parameter-shift gradient manually
def parameter_shift_grad_sv(params, gates, N, D, h, isRing):
    gradients = np.zeros_like(params)
    shift = torch.tensor(torch.pi / 2)
    for i in range(len(params)):
        params_forward = params.clone()
        params_backward = params.clone()
        params_forward[i] += shift
        params_backward[i] -= shift
        gradients[i] = (run_circuit_sv(params_forward,N,D,h, isRing) - run_circuit_sv(params_backward, N,D,h, isRing)) / 2
    return gradients

# Wrap into a PyTorch module
class QuantumCircuitTorch(torch.nn.Module):
    def __init__(self, theta, N, D, h, isRing=True):
        super().__init__()
        self.N = N
        self.D = D
        self.h = h
        self.gates = []
        self.isRing = isRing
        self.theta = torch.nn.Parameter(torch.tensor(theta, requires_grad=False))

    def forward(self):
        return QuantumFunction.apply(self.theta, self.gates, self.N, self.h, self.D, self.isRing)


In [ ]:
# =========================
# Resume-in-place utilities
# =========================
import os, csv, json
import torch

def _read_summary(summary_csv="maxcut_comparison_CPU/summary.csv"):
    with open(summary_csv, "r") as f:
        return list(csv.DictReader(f))

def _write_summary_header_if_missing(summary_csv="maxcut_comparison_CPU/summary.csv"):
    if not os.path.exists(summary_csv):
        with open(summary_csv, "w", newline="") as f:
            csv.writer(f).writerow(["run_id","method","N","depth","seed","theta_path","iter_csv"])

def _find_run_row(method: str, N: int, D: int, seed: int, summary_csv="maxcut_comparison_CPU/summary.csv"):
    """Return the LAST matching summary row (dict) or None."""
    if not os.path.exists(summary_csv):
        return None
    rows = _read_summary(summary_csv)
    cand = [r for r in rows if r["method"] == method and int(r["N"]) == N and int(r["depth"]) == D  and int(r["seed"]) == seed]
    return cand[-1] if cand else None

def _read_last_iter(iter_csv_path: str) -> int:
    """Return last iteration index in iter CSV, or -1 if file empty."""
    if not os.path.exists(iter_csv_path):
        return -1
    last = -1
    with open(iter_csv_path, "r") as f:
        r = csv.DictReader(f)
        for row in r:
            last = int(row["iter"])
    return last

def append_run_inplace(
    runlog: RunLog,
    *,
    method: str,
    N: int,
    D: int,
    seed: int,
    outdir: str = "maxcut_comparison_CPU",
    summary_csv: str = "maxcut_comparison_CPU/summary.csv",
):
    """
    Append new iterations to the *existing* run for (method, N, seed),
    and overwrite its final theta file. If not found, creates a new one.
    Returns the run_id and paths.
    """
    os.makedirs(outdir, exist_ok=True)
    _write_summary_header_if_missing(summary_csv)

    row = _find_run_row(method, N, D, seed, summary_csv)
    if row is None:
        # create a fresh entry (first time)
        import uuid
        run_id = str(uuid.uuid4())[:8]
        theta_path = os.path.join(outdir, f"theta_{run_id}.pt")
        iter_csv_path = os.path.join(outdir, f"iter_{run_id}.csv")
        # write header for new iter file
        with open(iter_csv_path, "w", newline="") as f:
            w = csv.writer(f); w.writerow(["iter","exp_value","best_result","iter_time"])
        # append summary row
        with open(summary_csv, "a", newline="") as f:
            w = csv.writer(f)
            w.writerow([run_id, method, N, D, seed, theta_path, iter_csv_path])
    else:
        run_id      = row["run_id"]
        theta_path  = row["theta_path"]
        iter_csv_path = row["iter_csv"]

    # figure out starting iteration index to continue
    start_iter = _read_last_iter(iter_csv_path) + 1

    # append new iteration rows with offset
    with open(iter_csv_path, "a", newline="") as f:
        w = csv.writer(f)
        for i, (e, b, t) in enumerate(zip(runlog.exp_values, runlog.best_results, runlog.iter_times)):
            w.writerow([start_iter + i, e, json.dumps(b), t])

    # overwrite final theta
    torch.save(runlog.final_theta.detach().cpu(), theta_path)

    return {"run_id": run_id, "theta_path": theta_path, "iter_csv": iter_csv_path, "start_iter": start_iter}

# =========================
# Resume runner (in-place)
# =========================
def benchmark_resume_inplace(
    method: str,
    edges,
    N: int,
    D: int,
    extra_iter_steps: int,
    *,
    seed: int,
    theta_fallback: torch.Tensor,   # used only if no prior run exists
    rank: int = 10,
    isRing: bool = True,
    lr_sv: float = 1e-2,
    lr_tr: float = 1e-2,
    shots: int | None = None,
    batch_size_ef: int = 200,
    batch_size_cs: int = 300,
    batch_size_ps: int = 200,
    outdir: str = "maxcut_comparison_CPU"
):
    """
    Continue training the SAME run (method, N, seed). If not existing, starts fresh.
    Appends new iterations to the existing CSV and overwrites final theta.
    """
    # set seeds
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

    # try loading last theta (if run exists)
    row = _find_run_row(method, N, D, seed, os.path.join(outdir, "summary.csv"))
    if row is not None and os.path.exists(row["theta_path"]):
        theta0 = torch.load(row["theta_path"], map_location="cpu").detach()
        print(f"[resume] Loaded previous theta for ({method}, N={N}, seed={seed}).")
    else:
        theta0 = theta_fallback.clone().detach()
        print(f"[resume] No previous run found; starting fresh for ({method}, N={N}, seed={seed}).")

    # build H and circuit (your helpers)
    hm   = create_hamiltonian(N, edges)
    h_qsv = make_hamiltonian(hm)
    h_tr  = build_qiskit_like_hamiltonian(hm)
    circuit = build_circuit(N, D, rank=rank, device="cpu", isRing=isRing)

    mm = {
        "tr_efficient": MeasureMethod.EFFICIENT_CONTRACTION,
        "tr_correct_sampling": MeasureMethod.CORRECT_SAMPLING,
        "tr_sampling": MeasureMethod.SAMPLING,
        "sv": "SV"
    }[method]
    if mm == MeasureMethod.CORRECT_SAMPLING:
      grad_common = dict(batches=batch_size_cs, shots=(shots or 10_000), D=D, isRing=isRing)
      runner = lambda: run_tr_vqe(mm, circuit, theta0, h_tr, extra_iter_steps, lr_tr, grad_common)
    elif mm == MeasureMethod.SAMPLING:
      grad_common = dict(batches=batch_size_ps, shots=(shots or 10_000), D=D, isRing=isRing)
      runner = lambda: run_tr_vqe(mm, circuit, theta0, h_tr, extra_iter_steps, lr_tr, grad_common)
    elif mm == "SV":
      ## def run_statevector_vqe(N: int, D: int, h, iter_steps: int, lr: float = 1e-2, theta0=None, isRing=True) -> RunLog:
      grad_common = dict(batches=100, shots=None, D=D, isRing=isRing)
      runner = lambda: run_statevector_vqe(N, D, h_qsv, extra_iter_steps, lr=lr_tr, theta0=theta0, isRing=isRing)
    else:
      grad_common = dict(batches=batch_size_ef, shots=(shots or 10_000), D=D, isRing=isRing)
      runner = lambda: run_tr_vqe(mm, circuit, theta0, h_tr, extra_iter_steps, lr_tr, grad_common)

    # run extra steps
    log: RunLog = runner()
    # append-in-place + overwrite theta
    info = append_run_inplace(log, method=method, N=N, D=D, seed=seed, outdir=outdir)
    print(f"[resume] Appended {len(log.exp_values)} iters starting from {info['start_iter']}.")
    return info, log


In [ ]:
# Load initial parameters
theta_init_by_seed = []
for i in range(10):
  a = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}.pt")
  b = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}_extra.pt")
  theta_init_by_seed.append(torch.cat([a,b]))

In [ ]:
with open("benchmark/experiment_grid.json", "r") as f:
    loaded = json.load(f)

# Convert lists back to tuples if you want
def convert_lists_to_tuples(obj):
    if isinstance(obj, list):
        # Check if list looks like an edge [(0,1), w]
        if len(obj) == 2 and isinstance(obj[0], list) and isinstance(obj[1], (int, float)):
            return (tuple(obj[0]), obj[1])
        else:
            return [convert_lists_to_tuples(x) for x in obj]
    if isinstance(obj, dict):
        return {k: convert_lists_to_tuples(v) for k, v in obj.items()}
    return obj

graph_by_seed = [convert_lists_to_tuples(entry) for entry in loaded]

In [ ]:
print(N_list)

[4, 8, 16, 24, 36, 52, 70, 90, 100]


In [ ]:
# Example: running over D and seeds for one specific graph_id and N
# target_graph_id = 2  # or loop over several ids
graph_lookup = {(g['N'], g['graph_id']): g['G'] for g in graph_by_seed}
# assert N == target_N  # optional sanity check
for N in [4,8,16,24]:
  for D in [3]:
      for seed in [0]:
          target_N = N
          g = graph_lookup[(target_N, seed)]
          N = max(max(u, v) for (u, v), _ in g) + 1  # or just use target_N
          param_size = 2 * D * N
          theta0 = theta_init_by_seed[seed][:param_size]
          for meth in ["tr_efficient", "tr_correct_sampling", "tr_sampling"]:
              print(f"[Graph {seed}] depth={D}, seed={seed}, N={N}")

              info, log = benchmark_resume_inplace(
                  method=meth,
                  edges=g,
                  N=N,
                  D=D,
                  extra_iter_steps=10,
                  seed=seed,
                  theta_fallback=theta0,
                  rank=10,
                  isRing=True,
                  lr_tr=1e-2,
                  shots=int(1e3),
                  outdir="maxcut_comparison_CPU",
                  batch_size_ef=400,
                  batch_size_cs=600,
                  batch_size_ps=1000,
              )

              torch.cuda.empty_cache()
              print("updated files:", info)
for N in [4,8,16]:
  for D in [3]:
      for seed in [0]:
          target_N = N
          g = graph_lookup[(target_N, seed)]
          N = max(max(u, v) for (u, v), _ in g) + 1  # or just use target_N
          param_size = 2 * D * N
          theta0 = theta_init_by_seed[seed][:param_size]
          for meth in ["sv"]:
              print(f"[Graph {seed}] depth={D}, seed={seed}, N={N}")

              info, log = benchmark_resume_inplace(
                  method=meth,
                  edges=g,
                  N=N,
                  D=D,
                  extra_iter_steps=10,
                  seed=seed,
                  theta_fallback=theta0,
                  rank=10,
                  isRing=True,
                  lr_tr=1e-2,
                  shots=int(1e3),
                  outdir="maxcut_comparison_CPU",
                  batch_size_ef=400,
                  batch_size_cs=600,
                  batch_size_ps=1000,
              )

              torch.cuda.empty_cache()
              print("updated files:", info)

[Graph 0] depth=3, seed=0, N=4
[resume] Loaded previous theta for (tr_efficient, N=4, seed=0).
Progress: [==========================>   ] 90% | ETA: 00:00 | Loss: 12.2527[resume] Appended 10 iters starting from 10.
updated files: {'run_id': '5c878e8f', 'theta_path': 'maxcut_comparison_CPU/theta_5c878e8f.pt', 'iter_csv': 'maxcut_comparison_CPU/iter_5c878e8f.csv', 'start_iter': 10}
[Graph 0] depth=3, seed=0, N=4
[resume] Loaded previous theta for (tr_correct_sampling, N=4, seed=0).
Progress: [==========================>   ] 90% | ETA: 00:00 | Loss: 11.6878[resume] Appended 10 iters starting from 10.
updated files: {'run_id': '9520ff0a', 'theta_path': 'maxcut_comparison_CPU/theta_9520ff0a.pt', 'iter_csv': 'maxcut_comparison_CPU/iter_9520ff0a.csv', 'start_iter': 10}
[Graph 0] depth=3, seed=0, N=4
[resume] No previous run found; starting fresh for (tr_sampling, N=4, seed=0).
Progress: [==========================>   ] 90% | ETA: 00:00 | Loss: 16.9254+0.0000j[resume] Appended 10 iters startin

/tmp/ipython-input-3869030728.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta = torch.nn.Parameter(torch.tensor(theta, requires_grad=False))
/tmp/ipython-input-3869030728.py:62: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  return grad_output *grad, None, None, None, None, None


[resume] Appended 10 iters starting from 0.
updated files: {'run_id': '57e098ad', 'theta_path': 'maxcut_comparison_CPU/theta_57e098ad.pt', 'iter_csv': 'maxcut_comparison_CPU/iter_57e098ad.csv', 'start_iter': 0}
[Graph 0] depth=3, seed=0, N=8
[resume] No previous run found; starting fresh for (sv, N=8, seed=0).
[resume] Appended 10 iters starting from 0.
updated files: {'run_id': '152caf8f', 'theta_path': 'maxcut_comparison_CPU/theta_152caf8f.pt', 'iter_csv': 'maxcut_comparison_CPU/iter_152caf8f.csv', 'start_iter': 0}
[Graph 0] depth=3, seed=0, N=16
[resume] No previous run found; starting fresh for (sv, N=16, seed=0).
[resume] Appended 10 iters starting from 0.
updated files: {'run_id': '4c7240c7', 'theta_path': 'maxcut_comparison_CPU/theta_4c7240c7.pt', 'iter_csv': 'maxcut_comparison_CPU/iter_4c7240c7.csv', 'start_iter': 0}


In [ ]:
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: Ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.6.0 which is incompatible.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
from optimization.optimization import minimize